In [2]:
import pandas as pd
from bs4 import BeautifulSoup

from openai import OpenAI
import json
import math

#importing the data

In [3]:
assist_df = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/full_dataset/skill_builder_data_corrected_collapsed.csv', encoding = "ISO-8859-1", low_memory=False)
pb_df = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/problem_bodies/ProblemBodies_23.csv', low_memory=False)
with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/pb_subset/keyid2idx_assist2009_pb_subset.json', 'r') as f:
  keyid2idx_pb_subset = json.load(f)
with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/full_dataset/keyid2idx_assist2009.json', 'r') as f:
  keyid2idx_full = json.load(f)

In [4]:
assist_df.head()

,Unnamed: 0,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,1,33022537,277618,64525,33139,51424,1,1,1,32454,...,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,2,33022709,277618,64525,33150,51435,1,1,1,4922,...,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,3,35450204,220674,70363,33159,51444,1,0,2,25390,...,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,4,35450295,220674,70363,33110,51395,1,1,1,4859,...,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,5,35450311,220674,70363,33196,51481,1,0,14,19813,...,3,4,124564,30060,NaN,65,0,0.0,3,3.0


In [5]:
pb_df.head()

,problem_id,problem_code,assistment_id,problem_set_id,problem_set_type,problem_set_name,curriculum,grade_or_subject,unit,link,...,attempts,correct_count,percent_correct,first_cwa,first_cwa_count,second_cwa,second_cwa_count,third_cwa,third_cwa_count,answer
0,1152985.0,PRA5FZU,780070.0,PSAQKFU,NaN,"8.5 Comparing Linear, Exponential, and Quadrat...",Textbook Curricula,Big Ideas Learning,HS: Purple Algebra I (2014),https://app.assistments.org/find/lv/lesson/328...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exponential
1,1164390.0,PRA5S4J,789764.0,PSAUDWH,NaN,Variables and Patterns Investigation 1 ACE,Textbook Curricula,Pearson (Prentice-Hall),CMP 3,https://app.assistments.org/find/lv/lesson/329...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1167935.0,PRA5V8Q,792776.0,PSAUK4X,NaN,Variables and Patterns Investigation 3 ACE,Textbook Curricula,Pearson (Prentice-Hall),CMP 3,https://app.assistments.org/find/lv/lesson/329...,...,33.0,21.0,63.6,18,3.0,15,2.0,0.3,1.0,12
3,1152694.0,PRA5FRU,779822.0,PSATNY9,NaN,9.7 Independent and Dependent Events,Textbook Curricula,Glencoe McGraw-Hill,Grade 7: Course 2 (2015),https://app.assistments.org/find/lv/lesson/328...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,1217278.0,PRA6768,832131.0,PSAVWDT,NaN,Covering and Surrounding ACE Investigation 4,Textbook Curricula,Pearson (Prentice-Hall),CMP 3,https://app.assistments.org/find/lv/lesson/329...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32


In [7]:
manual_skill_texts = {5:'Number Line',  \
                      102:'Number Line',  \
                      53:'Ordering Integers',  \
                      2:'Circle Graph',  \
                      37:'Circle Graph',  \
                      375:'Number Line',  \
                      10:'Table',  \
                      64:'Table',  \
                      9:'Stem and Leaf Plot',  \
                      14:'Stem and Leaf Plot',  \
                      74:'Subtraction Whole Numbers',  \
                      35:'Effect of Changing Dimensions of a Shape Prportionally',  \
                      69:'Unit Conversion Within a System',  \
                      173:'Choose an Equation from Given Information',  \
                      190:'Choose an Equation from Given Information',  \
                      193:'Choose an Equation from Given Information',  \
                      221:'Choose an Equation from Given Information',  \
                      321:'Pythagorean Theorem',
                      }

# Full dataset from skill names *only*

In [8]:
assist_df = assist_df.fillna('High School Math Concept or Skill')

skills = keyid2idx_full['concepts'].keys()
skill_names = assist_df[assist_df['skill_id'].isin(skills)]['skill_name'].unique()
i = 0
skill_texts = []
for skill in list(skills):
  sk_name = assist_df[assist_df['skill_id'] == skill]['skill_name'].unique()
  sk_name_int = assist_df[assist_df['skill_id'] == int(skill)]['skill_name'].unique()

  if sk_name.size > 0:
    skill_text = sk_name[0]
  elif sk_name_int.size > 0:
    skill_text = sk_name_int[0]
  else:
    skill_text = manual_skill_texts[int(skill)]


  skill_texts.append(skill_text)
skill_texts_stacked = ["INCORRECT " + s for s in skill_texts] + ["CORRECT " + s for s in skill_texts]

In [13]:
skill_texts_stacked


['INCORRECT Addition and Subtraction Positive Decimals',
 'INCORRECT Addition and Subtraction Fractions',
 'INCORRECT Multiplication and Division Integers',
 'INCORRECT Multiplication Fractions',
 'INCORRECT Addition Whole Numbers',
 'INCORRECT Addition and Subtraction Integers',
 'INCORRECT Rounding',
 'INCORRECT Congruence',
 'INCORRECT Ordering Integers',
 'INCORRECT Ordering Positive Decimals',
 'INCORRECT Order of Operations +,-,/,* () positive reals',
 'INCORRECT Absolute Value',
 'INCORRECT Conversion of Fraction Decimals Percents',
 'INCORRECT Unit Conversion Within a System',
 'INCORRECT Unit Rate',
 'INCORRECT Percent Of',
 'INCORRECT Multiplication and Division Positive Decimals',
 'INCORRECT Division Fractions',
 'INCORRECT Exponents',
 'INCORRECT High School Math Concept or Skill',
 'INCORRECT Finding Percents',
 'INCORRECT Divisibility Rules',
 'INCORRECT Least Common Multiple',
 'INCORRECT Ordering Fractions',
 'INCORRECT Ordering Integers',
 'INCORRECT Square Root',
 'I

In [15]:
from openai import OpenAI

with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/config/api_key.json', 'r') as f:
    api_key = json.load(f)['open_ai_api_key']

client = OpenAI(api_key=api_key)


In [16]:
# model_dims = [('text-embedding-3-small', 768), ('text-embedding-3-small', 1536), \
#  ('text-embedding-3-large', 768), ('text-embedding-3-large', 1536), ('text-embedding-3-large', 3072) \
#  , ('text-embedding-ada-002', 1536)]

model_dims = [('text-embedding-3-small', 768), ('text-embedding-3-small', 1536)]
import torch
for (model,emb_dim) in model_dims:
  print(model, emb_dim)
  skills_tensor = torch.empty((0, emb_dim))
  if model == 'text-embedding-ada-002':
    response_embeddings = client.embeddings.create(input = skill_texts_stacked, model=model).data
  else:
    response_embeddings = client.embeddings.create(input = skill_texts_stacked, model=model, dimensions=emb_dim).data

  for data in response_embeddings:
    emb = torch.tensor(data.embedding)
    emb = torch.reshape(emb, (1, emb_dim))
    skills_tensor = torch.cat((skills_tensor, emb), dim=0)
  torch.save(skills_tensor, '/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/emb/openai_'+ model + '_' +str(emb_dim) +  '.pt')


text-embedding-3-small 768
text-embedding-3-small 1536


In [12]:
skills_tensor.shape

torch.Size([246, 1536])

# Full dataset from skill names *only* (reflection method)

Done

In [ ]:
assist_df = assist_df.fillna('High School Math Concept or Skill')

skills = keyid2idx_full['concepts'].keys()
skill_names = assist_df[assist_df['skill_id'].isin(skills)]['skill_name'].unique()
i = 0
skill_texts = []
for skill in list(skills):
  sk_name = assist_df[assist_df['skill_id'] == skill]['skill_name'].unique()
  sk_name_int = assist_df[assist_df['skill_id'] == int(skill)]['skill_name'].unique()

  if sk_name.size > 0:
    skill_text = sk_name[0]
  elif sk_name_int.size > 0:
    skill_text = sk_name_int[0]
  else:
    skill_text = manual_skill_texts[int(skill)]


  skill_texts.append(skill_text)


In [ ]:
model_dims = [('text-embedding-3-small', 768), ('text-embedding-3-small', 1536), \
 ('text-embedding-3-large', 768), ('text-embedding-3-large', 1536), ('text-embedding-3-large', 3072) \
 , ('text-embedding-ada-002', 1536)]
import torch
for (model,emb_dim) in model_dims:
  print(model, emb_dim)
  skills_tensor = torch.empty((0, emb_dim))
  if model == 'text-embedding-ada-002':
    response_embeddings = client.embeddings.create(input = skill_texts_stacked, model=model).data
  else:
    response_embeddings = client.embeddings.create(input = skill_texts_stacked, model=model, dimensions=emb_dim).data

  for data in response_embeddings:
    emb = torch.tensor(data.embedding)
    emb = torch.reshape(emb, (1, emb_dim))
    skills_tensor = torch.cat((skills_tensor, emb), dim=0)
  for data in response_embeddings:
    emb = torch.tensor([x * -1 for x in data.embedding])
    emb = torch.reshape(emb, (1, emb_dim))
    skills_tensor = torch.cat((skills_tensor, emb), dim=0)

  torch.save(skills_tensor, '/content/drive/MyDrive/Education_Data/ProblemBodies2023_2.0/embeddings/assist2009_full_dataset_mirror/openai_'+ model + '_' +str(emb_dim) +  '.pt')


text-embedding-3-small 768
text-embedding-3-small 1536
text-embedding-3-large 768
text-embedding-3-large 1536
text-embedding-3-large 3072
text-embedding-ada-002 1536


# PB(ProblemBodies) subset dataset from skill names only

In [ ]:
assist_df = assist_df.fillna('High School Math Concept or Skill')

skills = keyid2idx_pb_subset['concepts'].keys()
skill_names = assist_df[assist_df['skill_id'].isin(skills)]['skill_name'].unique()
i = 0
skill_texts = []
for skill in list(skills):
  sk_name = assist_df[assist_df['skill_id'] == skill]['skill_name'].unique()
  sk_name_int = assist_df[assist_df['skill_id'] == float(skill)]['skill_name'].unique()

  if sk_name.size > 0:
    skill_text = sk_name[0]
  elif sk_name_int.size > 0:
    skill_text = sk_name_int[0]
  else:
    skill_text = manual_skill_texts[int(skill)]


  skill_texts.append(skill_text)
skill_texts_stacked = ["INCORRECT " + s for s in skill_texts] + ["CORRECT " + s for s in skill_texts]

In [ ]:
from openai import OpenAI

with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/config/api_key.json', 'r') as f:
    api_key = json.load(f)['open_ai_api_key']

client = OpenAI(api_key=api_key)
def get_embedding(text, model="text-embedding-3-small", emb_dim = 768):
  # text = text.replace("\n", " ")
  # return client.embeddings.create(input = [text], model=model).data[0].embedding
  return client.embeddings.create(input = [text], model=model, dimensions=emb_dim).data[0].embedding



In [ ]:
model_dims = [('text-embedding-3-small', 768), ('text-embedding-3-small', 1536), \
 ('text-embedding-3-large', 768), ('text-embedding-3-large', 1536), ('text-embedding-3-large', 3072) \
 , ('text-embedding-ada-002', 1536)]
import torch
for (model,emb_dim) in model_dims:
  print(model, emb_dim)
  skills_tensor = torch.empty((0, emb_dim))
  if model == 'text-embedding-ada-002':
    response_embeddings = client.embeddings.create(input = skill_texts_stacked, model=model).data
  else:
    response_embeddings = client.embeddings.create(input = skill_texts_stacked, model=model, dimensions=emb_dim).data

  for data in response_embeddings:
    emb = torch.tensor(data.embedding)
    emb = torch.reshape(emb, (1, emb_dim))
    skills_tensor = torch.cat((skills_tensor, emb), dim=0)

  torch.save(skills_tensor, '/content/drive/MyDrive/Education_Data/ProblemBodies2023_2.0/embeddings/assist2009_pb_subset/openai_'+ model + '_' +str(emb_dim) +  '.pt')


text-embedding-3-small 768
text-embedding-3-small 1536
text-embedding-3-large 768
text-embedding-3-large 1536
text-embedding-3-large 3072
text-embedding-ada-002 1536


# Question Embeddings [link text](https://)(PB Subset)

In [ ]:
pb_subset_df = pd.read_csv('/content/drive/MyDrive/Education_Data/ProblemBodies2023_2.0/subset_problembodies_preprocessed/pb_assist2009_subset_preprocessed.csv')

In [ ]:
pb_subset_df = pb_subset_df.fillna('High School Math Question')

skill_texts = pb_subset_df['problem_body'].tolist()

skill_texts_stacked = ["INCORRECT " + s for s in skill_texts] + ["CORRECT " + s for s in skill_texts]

In [ ]:

from openai import OpenAI

with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/config/api_key.json', 'r') as f:
    api_key = json.load(f)['open_ai_api_key']

client = OpenAI(api_key=api_key)

model_dims = [('text-embedding-3-small', 768), ('text-embedding-3-small', 1536), \
 ('text-embedding-3-large', 768), ('text-embedding-3-large', 1536), ('text-embedding-3-large', 3072) \
 , ('text-embedding-ada-002', 1536)]
import torch

chunk_size = 2048
skill_texts_stacked_chunks = [skill_texts_stacked[i:i + chunk_size] for i in range(0, len(skill_texts_stacked), chunk_size)]
for (model,emb_dim) in model_dims:
  print(model, emb_dim)
  skills_tensor = torch.empty((0, emb_dim))
  for chunk in skill_texts_stacked_chunks:
    if model == 'text-embedding-ada-002':
      response_embeddings = client.embeddings.create(input = chunk, model=model).data
    else:
      response_embeddings = client.embeddings.create(input = chunk, model=model, dimensions=emb_dim).data
    for data in response_embeddings:
      emb = torch.tensor(data.embedding)
      emb = torch.reshape(emb, (1, emb_dim))
      skills_tensor = torch.cat((skills_tensor, emb), dim=0)

  torch.save(skills_tensor, '/content/drive/MyDrive/Education_Data/ProblemBodies2023_2.0/embeddings/assist2009_questions_pb_subset/openai_'+ model + '_' +str(emb_dim) +  '.pt')


text-embedding-3-small 768
text-embedding-3-small 1536
text-embedding-3-large 768
text-embedding-3-large 1536
text-embedding-3-large 3072
text-embedding-ada-002 1536


In [ ]:
# prompt: divide up skill_texts_stacked into chunks of smaller than 2048

chunk_size = 2048
skill_texts_stacked_chunks = [skill_texts_stacked[i:i + chunk_size] for i in range(0, len(skill_texts_stacked), chunk_size)]


12282

In [ ]:
for row in skill_texts_stacked:
  print(row)
  response_embeddings = client.embeddings.create(input = row, model=model, dimensions=emb_dim).data


Streaming output truncated to the last 5000 lines.
CORRECT Below is a list of numbers. [48, 76, 84, 23, 45, 21, 24, 34, 65, 36, 41, 56] What is the median number in this list?
CORRECT Which of the following statements explains the correct method to solve the equation -4 +  7y = 24 ?
CORRECT Steve took a survey of the students in his school to find each student's favorite baked goods.  He recorded the results in order to make the circle graph below. Favorite Baked Goods Cakes 40% Pies 15% Cookies 20% Brownies 25% According to this graph, if 160 students have Cookies as their favorite baked goods, then how many students in total did Steve survey?
CORRECT Which one of the flat nets represent the given Cube figure?
CORRECT The perimeter of the paralellogram below is 60 in.  What is the length of side BC? 9 in. Note: The image is NOT drawn to scale.
CORRECT Find the value of a that makes the fraction equivalent. a 30 = 11 5
CORRECT Identify the type of the colored angle in the following fig

In [ ]:
pb_subset_df

NameError: name 'pb_subset_df' is not defined